<h1>콘텐츠 기반 필터링 - Contents Based Filtering</h1>

콘텐츠 기반 필터링은 말그대로 콘텐츠 에 기반한 속성 유사도를 가지고 추천을 해주는 그런 알고리즘이다.

영화 같은 경우를 예를 들자면 Arrival(우리나라에선 컨택트로 개봉) 이나 프로메테우스 같이 좋은 평점을 받았다.

보면 장르는 SF 드라마, 등. 여기서 적지 않은 비평가들이 적은 다양한 속성, 마케팅 메세지 들 등이 있다.

이것들을 분석해서 비슷한 SF 액션에, 같은 감독, 출연자는 살짝 다르지만 키워드 등을 분석해서 추천하는 것이다.

이런것들을 다 비교를 해보니까 이게 맞아떨어지는구나, 이 사용자는 두개의 컨텐츠에 대한 다양한 요소들을 비교해보니까

여기에 있는 요소들하고 맞아 떨어지는구나해서 사용자가 블레이드러너를 추천 받을 수 있는 것이다.

그러는 와중에 사용자 선호 프로파일이 만들어진다.

<img src="./img/ContBasedFil1.png">

<h2>콘텐츠 기반 필터링 기법</h2>

이 콘텐츠 기반 필터링을 약간 구닥다리 아냐? 라고 생각할 수 있지만 의외로 많이 쓰인다.

근데 문제가 뭐냐면 description을 보면 너무 마케팅 메세지가 들어가 있는 것이다.

영화에 가장 큰 문제는 예고편에서 반했는데, 혹은 온라인에서 배급사측에서 만들어논 마케팅 메세지를 보고 갔는데 실제로는 다르다던가.

너무 마케팅적인 메세지에 의해 정보가 명확하게 되지 않은 단점이 있음에도 불구하고 콘텐츠 기반 필터링 방식은 자주 활용되는 방식이다.

음악같은 경우는 수십명 수백명등의 전문가들이 공정하게 평가한 방대한 콘텐츠 정보를 가지고 있는 사이트도 있다.

일반적으로 영화 구성 콘텐츠를 텍스트해서 피처 벡터화 해버리고 코사인 유사도를 구하여 유사도 및 평점(좋게받은)에 따른 영화 추천을 한다.

<img src="./img/ContBasedFil2.png">

<h2>콘텐츠 기반 필터링 실습 - TMDB 5000</h2>

TMDB는 미국의 유명한 영화 평점 사이트이다. IMBD는 굉장히 유명해서 데이터 분석가들한테 이러한 정보들을 오픈을 한다.

거기에서 5천개만 압축을 한거고 캐글에서 다운 받을 것임.

<img src="./img/ContBasedFilTMDB.png">

<h2>콘텐츠 기반 필터링 실습</h2>

콘텐츠 기반 필터링 구현 프로세스

1. 톤텐츠에 대한 여러 텍스트 정보들을 피처 벡터화 할것이다.

2. 코사인 유사도로 콘텐츠별 유사도를 계싼할 것이다. 

3. 콘텐츠 별로 가중 평점을 계산한다.

4. 유사도가 높은 콘텐츠 중에 평점이 좋은 순으로 추천할 것이다.

<img src="./img/ContBasedFildo.png">

데이셋인 movies 데이터만 쓸 것이다. 

영화 각각에 대한 메타 데이터가 들어가 있다.

### 9.1 컨텐츠 기반 필터링 실습 – TMDB 5000 Movie Dataset

In [1]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies =pd.read_csv('./tmdb-5000-movie-dataset/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


데이터를 로딩해서 앞에 한건만 보면 컬럼들을 쭉 볼 수 있는데, 가져갈거는 title, genres, vote_average, vote_count만 거의 쓸 것이ㅏㄷ.

좀더 가져간다면 overview, popularity, keiwords 정도가 있을 것이다.

In [2]:
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]


장르만 한번 다시 확인해보자.

In [3]:
pd.set_option('max_colwidth', 100)
movies_df[['genres','keywords']][:1]


,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


장르하고 키워드를 같이 써서 콘텐츠가 유사하구나 이렇게 판단해도 되고, 유형만 알면 되니까 간략하게 하기 위해서 장르만 가지고 콘텐츠 유사도를 판단해보겠음.

근데 이제 안해본걸 할것임. 


In [4]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   overview      4800 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


장르가 다 텍스트이다. 여러개의 유형이 있는데 장르만 뽑아내고 싶다. 텍스트 문서들을 어떻게 파싱을 해서 장르라는 그 집합체로 가져갈 수 있을까를 실습할 것임. 

텍스트 문서를 객체로 만들 수 있다.

**텍스트 문자 1차 가공. 파이썬 딕셔너리 변환 후 리스트 형태로 변환**

그것을 가능하게 하는게 파이썬에 ast 패키지가 있다. literal_eval이라는 함수를 쓰게 되면 텍스트를 자동으로 파싱을 해서 거기에 맞는 파이썬의 객체로 만들어준다.

첫번째는 파이썬의 딕셔너리로 변환을 해주는 것

In [5]:
from ast import literal_eval

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [6]:
movies_df['genres'].head(1)

0    [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...
Name: genres, dtype: object

이 중에 딕셔너리에 name 키를 가지고 있는 벨류 (action)만 추출을 해볼 것임.

리스트 컴프리헨션으로 감쌀 것임. apply lambda로 하면 저게 다 들어올 것인데, 그 개별 딕셔너리 중에 'name'만 가지고 있는 것을 추출할 것임. 

키워드는 안쓸 건데, 관심있는 사람은 장르하고 섞어서 코사인 유사도를 만들어봐도 된다.

In [7]:
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


위에 결과는 리스트이다. 0번 영화는 이렇게 여러가지 장르를 가지게 되는 것임.

**장르 콘텐츠 필터링을 이용한 영화 추천. 장르 문자열을 Count 벡터화 후에 코사인 유사도로 각 영화를 비교**

**장르 문자열의 Count기반 피처 벡터화**

In [8]:
type(('*').join(['test', 'test2']))

str

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환. 
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


컬럼이 276개이다.

**장르에 따른 영화별 코사인 유사도 추출**

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])


(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


영화가 4803개니까 당연히 이렇게 나올 것임.

그 다음에 뭐할거냐면 argsort를 또 사용할거다. 왜 계속 했었나? 반환값이 넘파이일 경우에 

실제 데이터 추출은 데이터프레임에 해야될 경우 넘파이에 argsort를 자주 써서 인덱스를 가져오는 연산을 할 건데, 

먼저 이걸 하기전에 cosine_similarity는 2차원 배열로 반환이 되었다. 예를 들어서 [[1,0.1,0.05,0.3],[...][...][...][...]] 이런식으로 되었다.
               0   1     2    3
0번 인덱스 -> [1, 0.01, 0.05,0.3]
1번 인덱스 -> []
2번 인덱스 -> []

genre_sim_sorted_ind의 경우도 2차원 형식으로 들어간 것이다. 0번 영화에 대해서 코사인 유사도로 가장 유사한 값을 가지고 있는

영화가 인덱스로 나온 것임.

In [11]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0 3494  813 ... 3038 3037 2401]]


**특정 영화와 장르별 유사도가 높은 영화를 반환하는 함수 생성**

In [12]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    #title_name은 기준영화, top_n은 기준영화와 유사한 10개를 뜻하는 것이다.    
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]


In [13]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average']]

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


첫번째 나온 것은

print(similar_indexes)

이다.

약간 이상한 경우가 있다. 평점 0.0짜리도 있다.

GoodFellas 같이 갱스터 영화에 감독도 같은데 뒷 순위로 밀린 경우도 있다. 

그래서 비슷한 컨텐츠이지만 평점도 높으면 좋으므로 그렇게 메커니즘을 바꿀 것이다.

**평점이 높은 영화 정보 확인**

유의할 것이 있다. vote_count가 굉장히 적은 평점 데이터를 가지고 있는 정보는 신뢰하기 어렵다.

In [14]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


그러므로 가중치를 부여할 것임

**평가 횟수에 대한 가중치가 부여된 평점(Weighted Rating) 계산  
         가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C**
         
■ v: 개별 영화에 평점을 투표한 횟수
■ m: 평점을 부여하기 위한 최소 투표 횟수
■ R: 개별 영화에 대한 평균 평점.
■ C: 전체 영화에 대한 평균 평점

투표 횟수 정도에서 가중치를 부여하는 것.<br>
예를 들어 1000번 투표를 한 사람이 있으면 상위 60%를 봐도 300회 밖에 안된다고 쳤을때<br>

1000/(1000+300) *R

반대로 10번이라고 하면 10/10+300*R. 확 줄 수 있다.

C는 전반적으로 평점에 후하거나 짠 사람에 대해 밸런스를 조절하는 것임.

In [16]:
C = movies_df['vote_average'].mean()  
m = movies_df['vote_count'].quantile(0.6)
print('C 전체 영화에 대한 평균 평점:',round(C,3), 'm 상위 0.6% 투표 횟수:',round(m,3))

C 전체 영화에 대한 평균 평점: 6.092 m 상위 0.6% 투표 횟수: 370.2


**이제 함수를 하나 만들 것임.**

In [17]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) 


weighted_vote를 내림차순으로 10개만 추출해보자.

In [18]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',
                                                                          ascending=False)[:10]


,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


위 결과를 보니 어느정도 객관적인 수준에 이르렀다고 볼 수 있다.

이제 앞에 함수에 약간의 수정을 가할 것이다.

top_n을 10개 뽑으라고 했었지만 2배를 하여 좀 더 여유있게 뽑을 것임.

그리고 기준영화 index는 제외할 것임.이렇게 해서 데이터들이 추출이 됐으면

이 중에 top_n만큼 추출을 하는데 weighted_vote로 내림차순으로 추출할 것이다.

In [19]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
# 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]


,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427


갱스터 무비들이 많이 추출됐다. 유사도가 조금 낮더라도 평점순으로 정렬이되어서 훨씬 더 사용자가 만족할 수 있는 방식으로 추출되었음을 알 수 있다. 다음 시간에는 협업 필터링을 실습해볼 것임.